## Создание индекса FAISS для бота

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

# Получаем абсолютный путь к корневой директории проекта (директория выше текущей)
root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Добавляем корневую директорию в sys.path
if root_path not in sys.path:
    sys.path.append(root_path)

In [3]:
import torch
from utils import parse_yaml
from models.clap_encoder import CLAP_Encoder
import faiss
from faiss import write_index, read_index
from utils import find_nearest_embedding

2024-04-15 01:27:49.735569: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-15 01:27:49.735600: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-15 01:27:49.736591: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-15 01:27:49.741078: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-15 01:27:50.219912: W tensorflow/compiler/tf2

In [4]:
SS_CONFIG_PATH = '../config/audiosep_base.yaml'
CLAP_CKPT_PATH = '../checkpoint/music_speech_audioset_epoch_15_esc_89.98.pt'

In [5]:
device = torch.device('cuda')
configs = parse_yaml(SS_CONFIG_PATH)

query_encoder = CLAP_Encoder(pretrained_path = CLAP_CKPT_PATH).eval().to(device)

2024-04-15 01:27:53,304 - INFO - Loading HTSAT-base model config.
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use 

In [6]:
desed_classes = ['vacuum cleaner',
                 'frying',
                 'dishes',
                 'blender',
                 'speech',
                 'dog',
                 'alarm bell ringing',
                 'running water',
                 'electric shaver toothbrush',
                 'cat']

desed_ckpt_path = 'desed.ckpt'

musdb_classes = ['drums', 'bass', 'vocal']
musdb_ckpt_path = 'musdb.ckpt'

# Объединяем классы и пути в один список с таплами (класс, путь)
combined_classes = [(cls, desed_ckpt_path) for cls in desed_classes] + \
                   [(cls, musdb_ckpt_path) for cls in musdb_classes]

# Генерируем эмбеддинги для всех классов
all_classes = [cls for cls, _ in combined_classes]
all_embeddings = query_encoder.get_query_embed(modality='text', text=all_classes).cpu()

# Создаем индекс FAISS
index = faiss.IndexHNSWFlat(512, 32)
index.add(all_embeddings)

In [7]:
query_embedding = query_encoder.get_query_embed(modality='text', text=['bark']).cpu()
predicted_class, ckpt_path, distance = find_nearest_embedding(combined_classes, query_embedding, index, threshold=1.0)
print(predicted_class, ckpt_path, distance)

dog desed.ckpt 0.23877218


In [8]:
index_path = "musdb_desed.idx"
write_index(index, index_path)
index_from_disk = read_index(index_path)
query_embedding = query_encoder.get_query_embed(modality='text', text=['bark']).cpu()
predicted_class, ckpt_path, distance = find_nearest_embedding(combined_classes, query_embedding, index_from_disk, threshold=1.0)
print(predicted_class, ckpt_path, distance)

dog desed.ckpt 0.23877218
